In [1]:
import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
import numpy as np
np.random.seed(0)
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [2]:
# Hyper-parameters 
input_size = 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [3]:
# MNIST dataset (images and labels)
train_dataset = torchvision.datasets.MNIST(root='data/', 
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='data/', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader (input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [4]:
# Logistic regression model
model = nn.Linear(input_size, num_classes).to(device)

# Loss and optimizer
# nn.CrossEntropyLoss() computes softmax internally
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [5]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Reshape images to (batch_size, input_size)
        images = images.reshape(-1, 28 * 28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i + 1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 2.2451
Epoch [1/5], Step [200/600], Loss: 2.1384
Epoch [1/5], Step [300/600], Loss: 2.0546
Epoch [1/5], Step [400/600], Loss: 2.0055
Epoch [1/5], Step [500/600], Loss: 1.8755
Epoch [1/5], Step [600/600], Loss: 1.7942
Epoch [2/5], Step [100/600], Loss: 1.7945
Epoch [2/5], Step [200/600], Loss: 1.7089
Epoch [2/5], Step [300/600], Loss: 1.6542
Epoch [2/5], Step [400/600], Loss: 1.5113
Epoch [2/5], Step [500/600], Loss: 1.5523
Epoch [2/5], Step [600/600], Loss: 1.4825
Epoch [3/5], Step [100/600], Loss: 1.3727
Epoch [3/5], Step [200/600], Loss: 1.4064
Epoch [3/5], Step [300/600], Loss: 1.3774
Epoch [3/5], Step [400/600], Loss: 1.2686
Epoch [3/5], Step [500/600], Loss: 1.2866
Epoch [3/5], Step [600/600], Loss: 1.3476
Epoch [4/5], Step [100/600], Loss: 1.2798
Epoch [4/5], Step [200/600], Loss: 1.2593
Epoch [4/5], Step [300/600], Loss: 1.1866
Epoch [4/5], Step [400/600], Loss: 1.0743
Epoch [4/5], Step [500/600], Loss: 1.1267
Epoch [4/5], Step [600/600], Loss:

In [6]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28 * 28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

    print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the model on the 10000 test images: 82 %


In [7]:
# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')